In [48]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from artemis.interactions_methods.model_agnostic import FriedmanHStatisticMethod, GreenwellMethod, SejongOhMethod
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from artemis.utilities.domain import InteractionMethod
from sklearn.neural_network import MLPRegressor
from artemis.additivity import AdditivityMeter
from sklearn.model_selection import train_test_split
from copy import copy

$$
Y = B_1 * X_1 + B_2 * X2 + B_3 * X_3 + B_4 (X1 * X2) + \epsilon
$$

In [49]:
N = 100
lower = -5
upper = 5

betas = np.array([3, -7, 10, 2])

X = pd.DataFrame(np.random.uniform(lower, upper, size=(N, 3)), columns=["x1", "x2", "x3"])
eps = np.random.normal(size=(N,))
y = X.apply(lambda row: np.dot(np.append(row, row["x1"]*row["x2"]), betas), axis=1)
y = y + eps


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [132]:
models = [("linear", LinearRegression()), ("random_forest", RandomForestRegressor(max_depth=25, max_features=2)), ("neural_network", MLPRegressor(hidden_layer_sizes=(5, 2), max_iter=30000))]

In [133]:
for name_model, model in models:
    model.fit(X_train, y_train)

/Users/pawel_fijalkowski/Praca inżynierska/feature-interactions-explanations/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [53]:
result = list()
methods_model_agnostic = [
        (InteractionMethod.H_STATISTIC, FriedmanHStatisticMethod()),
        (InteractionMethod.H_STATISTIC + " NO_NORMALIZATION", FriedmanHStatisticMethod(normalized=False)),
        (InteractionMethod.VARIABLE_INTERACTION, GreenwellMethod()),
        (InteractionMethod.PERFORMANCE_BASED, SejongOhMethod())
]

for name_model, model in tqdm(models):

    for name_method, method in tqdm(methods_model_agnostic):

        method_copy = copy(method)

        if method_copy.method == InteractionMethod.PERFORMANCE_BASED:
            method_copy.fit(model, X_train, y_true=y_train)
        else:
            method_copy.fit(model, X_train)

        result.append({"model": name_model, "method": name_method, "ovo": method_copy.ovo})


100%|██████████| 3/3 [00:42<00:00, 14.13s/it]


In [54]:
result = pd.DataFrame.from_records(result)

Due to its nature, linear model is unable to detect feature interactions.
Therefore, as a sanity-check of interaction method correctness, we check if all methods have interaction values that are close to 0.

In [55]:
THRESHOLD = 10**(-6)

In [56]:
for method_name, method in methods_model_agnostic:

    print(f"\n\nMethod: {method_name}")
    df = result.loc[(result["model"] == "linear") & (result["method"] == method_name), "ovo"].iloc[0].copy()
    df["is_below_threshold"] = np.abs(df[method.method]) < THRESHOLD
    print(df[["Feature 1", "Feature 2", "is_below_threshold"]])
    print("-"*50)



Method: Friedman H-statistic Interaction Measure
  Feature 1 Feature 2  is_below_threshold
0        x1        x2                True
1        x2        x3                True
2        x1        x3                True
--------------------------------------------------


Method: Friedman H-statistic Interaction Measure NO_NORMALIZATION
  Feature 1 Feature 2  is_below_threshold
0        x2        x3                True
1        x1        x3                True
2        x1        x2                True
--------------------------------------------------


Method: Greenwell Variable Interaction Measure
  Feature 1 Feature 2  is_below_threshold
0        x2        x3                True
1        x1        x3                True
2        x1        x2                True
--------------------------------------------------


Method: Sejong Oh Performance Based Interaction Measure
  Feature 1 Feature 2  is_below_threshold
0        x2        x3               False
1        x1        x3            

As we can see, `Sejong Oh Performance Based Interaction` falsely detects interactions in the linear model. It suggests its high limitations.

In [57]:
for method_name, _ in methods_model_agnostic:

    print(f"\n\nMethod: {method_name}")
    df = result.loc[(result["model"] == "random_forest") & (result["method"] == method_name), "ovo"].iloc[0].copy()
    print(df)
    print("-"*50)



Method: Friedman H-statistic Interaction Measure
  Feature 1 Feature 2  Friedman H-statistic Interaction Measure
0        x1        x2                                  0.098853
1        x2        x3                                  0.019067
2        x1        x3                                  0.013937
--------------------------------------------------


Method: Friedman H-statistic Interaction Measure NO_NORMALIZATION
  Feature 1 Feature 2  Friedman H-statistic Interaction Measure
0        x1        x2                                 44.884028
1        x2        x3                                 34.093386
2        x1        x3                                 29.344580
--------------------------------------------------


Method: Greenwell Variable Interaction Measure
  Feature 1 Feature 2  Greenwell Variable Interaction Measure
0        x1        x2                                5.235090
1        x1        x3                                3.224117
2        x2        x3           

In case of ` RandomForest`, both `Friedman H-statistic` (both normalized and un-normalized) and `Greenwell Variable interaction` seem to correctly capture `x1 - x2` non-additive influence (interaction). Again, `Sejong Oh Performance Based Interaction` falsely claims that `x2 - x3` interaction is the strongest. This suggests it's high limitations to correctly detect feature interactions.

In [58]:
for method_name, _ in methods_model_agnostic:

    print(f"\n\nMethod: {method_name}")
    df = result.loc[(result["model"] == "neural_network") & (result["method"] == method_name), "ovo"].iloc[0].copy()
    print(df)
    print("-"*50)



Method: Friedman H-statistic Interaction Measure
  Feature 1 Feature 2  Friedman H-statistic Interaction Measure
0        x1        x2                                  0.400538
1        x1        x3                                  0.031800
2        x2        x3                                  0.030689
--------------------------------------------------


Method: Friedman H-statistic Interaction Measure NO_NORMALIZATION
  Feature 1 Feature 2  Friedman H-statistic Interaction Measure
0        x1        x2                                 94.825400
1        x2        x3                                 38.782525
2        x1        x3                                 37.367775
--------------------------------------------------


Method: Greenwell Variable Interaction Measure
  Feature 1 Feature 2  Greenwell Variable Interaction Measure
0        x1        x2                                7.776908
1        x1        x3                                4.004394
2        x2        x3           

In case of `MLP`, all of the methods correctly capture `x1 - x2` as the highest interaction. `Friedman H-statistic` (normalized and un-normalized) and `Greenwell Variable interaction` values are negligible for pairs different to the actual interaction (`x1 - x2`). It may suggest their capability to correctly capture feature interactions. For well-performing model such as `MLP`, performance-based `Sejong Oh Performance Based Interaction` correctly indicates `x1 - x2` as the most relevant. This may suggest, that for well-performing models such as neural networks, performance-based methods may produce good results. Nevertheless, non-existing interaction values between `x1 - x3` and `x2 - x3` have high interaction values.

In [59]:
additivity_meter = AdditivityMeter()

In [60]:
additivity = list()
for name_model, model in tqdm(models):
    additivity_meter.fit(model, X)
    additivity.append({"model": name_model, "additivity": additivity_meter.additivity_index})

100%|██████████| 3/3 [00:00<00:00,  4.82it/s]


In [61]:
pd.DataFrame.from_records(additivity)

,model,additivity
0,linear,1.000000
1,random_forest,0.923338
2,neural_network,0.785224


As we can see, the more complex the model is, the less additive nature it has (with linear model having perfect 1.0 additivity). This measure clearly depicts how much variance in each model is explained by interactions between features.

Now, let's consider the potential impact of the feature interaction detection on the performance of the analysed models. Let's assume that using feature interaction methods we improved our understanding of the underlying problem and  discovered the interaction between `x1 - x2` features.

In [97]:
mse = lambda x, y : (np.square(x - y)).mean(axis=0)

In [134]:
performance_baseline = pd.DataFrame.from_records([{"model": name, "MSE - test": mse(model.predict(X_test), y_test)} for name, model in models])
performance_baseline

,model,MSE - test
0,linear,459.999990
1,random_forest,314.455318
2,neural_network,196.367686


Now, as a part of feature engineering process, let's add the detected interaction as a new feature.

In [64]:
X_interaction_detected = X.copy()
X_interaction_detected["x1*x2"] = X_interaction_detected.apply(lambda row: row["x1"] * row["x2"], axis=1)
X_interaction_detected

,x1,x2,x3,x1*x2
0,4.928706,-1.314451,-1.389005,-6.478543
1,-3.126601,-1.526824,2.481886,4.773768
2,-2.021114,-2.126540,1.693999,4.297979
3,-4.122396,-1.459970,-1.925510,6.018575
4,-3.812560,-1.830344,-4.327182,6.978295
...,...,...,...,...
95,-2.349113,-1.862272,-4.246464,4.374689
96,-3.839524,4.116100,3.722307,-15.803861
97,0.985637,-1.327644,2.217088,-1.308575
98,-3.694322,4.789777,-0.149619,-17.694979


In [65]:
X_train_int, X_test_int, y_train_int, y_test_int = train_test_split(X_interaction_detected, y, test_size=0.3, random_state=42)

In [129]:
models_int = [("linear", LinearRegression()), ("random_forest", RandomForestRegressor(max_depth=25, max_features=2)), ("neural_network", MLPRegressor(hidden_layer_sizes=(5, 2), max_iter=30000))]

In [130]:
for _, model in models_int:
    model.fit(X_train_int, y_train_int)

In [131]:
performance_interaction = pd.DataFrame.from_records([{"model": name, "MSE - test": mse(model.predict(X_test_int), y_test_int)} for name, model in models_int])
performance_interaction

,model,MSE - test
0,linear,1.500586
1,random_forest,233.257582
2,neural_network,2.880819


Comparing `performance_baseline` and `performance_interaction`, we can clearly see that adding detected interaction to the data significantly improved the performance of all the models. Of course, in this case target (`y`) becomes linear function of the features and so `MSE` of the `linear` model is close to zero. It suggests that discovering interaction in the data not only improves the understanding and explainability of the model/problem, but also may improve the model performance.

Above example raises important question about how in real life scenario should the `artemis` user know what function best represents the interaction between pair of features. In our case, data was generated from the known model, and so we know that non-additivity is captured within multiplication operation between `x1` and `x2`. Currently, automatic function search is not implemented, but creating such a functionality and combining it with `artemis` lays foundation for future work.